# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [207]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [208]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [209]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [210]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [211]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [212]:
# TO-DO: Create a Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [213]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [214]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query1 = "CREATE TABLE IF NOT EXISTS session_history"
query1 = query1 + "(artist text, song text, length float, sessionId int, itemInSession int, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query1)
except Exception as e:
    print(e)                   

In [215]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query1 = "INSERT INTO session_history (artist, song, length, sessionId, itemInSession)"
        query1 = query1 + "VALUES (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query1, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

In [216]:
# I convert row data to data frame to verify nosql queries for following sections.
def read_row_data():
    df = pd.read_csv(file)
    return df

#### Do a SELECT to verify that the data have been inserted into each table

In [217]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query1 = "select * from session_history where sessionId = 338 and itemInSession=4"

try:
    rows = session.execute(query1)
except Exception as e:
    print(e)

for row in rows:
    print("\033[1m"+"Artist:"+"\033[0m", row.artist, 
          "\033[1m"+"Song:"+"\033[0m", row.song, 
          "\033[1m"+"Length:"+"\033[0m", row.length, 
          "\033[1m"+"Session ID:"+"\033[0m", row.sessionid, 
          "\033[1m"+"Item In Session:"+"\033[0m", row.iteminsession
         )

Artist: Faithless Song: Music Matters (Mark Knight Dub) Length: 495.30731201171875 Session ID: 338 Item In Session: 4


### VERIFYING QUERY 1

In [218]:
df = read_row_data()
df = df[(df.sessionId==338) & (df.itemInSession==4)]
df = df[['artist', 'song', 'length', 'sessionId', 'itemInSession']]
df

,artist,song,length,sessionId,itemInSession
444,Faithless,Music Matters (Mark Knight Dub),495.3073,338,4


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [219]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query2 = "CREATE TABLE IF NOT EXISTS user_history"
query2 = query2 + "(artist text, song text, sessionId int, itemInSession int, firstName text, lastName text, userId int, PRIMARY KEY ((userId,SessionId),itemInSession))"
try:
    session.execute(query2)
except Exception as e:
    print(e)    

In [220]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query2 = "INSERT INTO user_history (artist, song, sessionId, itemInSession, firstName, lastName, userId)"
        query2 = query2 + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query2, (line[0], line[9], int(line[8]), int(line[3]), line[1], line[4], int(line[10])))

In [221]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query2 = "select * from user_history where userId = 10 and SessionId=182"

try:
    rows = session.execute(query2)
except Exception as e:
    print(e)

for row in rows:
    print("\033[1m"+"Artist:"+"\033[0m", row.artist, 
          "\033[1m"+"Song:"+"\033[0m", row.song,
          "\033[1m"+"First Name:"+"\033[0m",row.firstname,
          "\033[1m"+"Last Name:"+"\033[0m", row.lastname, 
          "\033[1m"+"Item In Session:"+"\033[0m", row.iteminsession
         )

Artist: Down To The Bone Song: Keep On Keepin' On First Name: Sylvie Last Name: Cruz Item In Session: 0
Artist: Three Drives Song: Greece 2000 First Name: Sylvie Last Name: Cruz Item In Session: 1
Artist: Sebastien Tellier Song: Kilometer First Name: Sylvie Last Name: Cruz Item In Session: 2
Artist: Lonnie Gordon Song: Catch You Baby (Steve Pitron & Max Sanna Radio Edit) First Name: Sylvie Last Name: Cruz Item In Session: 3


### VERIFYING QUERY 2

In [222]:
df = read_row_data()
df = df[(df.userId==10) & (df.sessionId==182)]
df = df[['artist', 'song', 'firstName', 'lastName', 'itemInSession']]
df

,artist,song,firstName,lastName,itemInSession
4704,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz,0
4705,Three Drives,Greece 2000,Sylvie,Cruz,1
4706,Sebastien Tellier,Kilometer,Sylvie,Cruz,2
4707,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz,3


In [223]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query3 = "CREATE TABLE IF NOT EXISTS listen_history"
query3 = query3 + "(userId int, firstName text, lastName text, song text, PRIMARY KEY (song, userID))"
try:
    session.execute(query3)
except Exception as e:
    print(e)              

In [224]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query3 = "INSERT INTO listen_history (userId, firstName, lastName, song)"
        query3 = query3 + "VALUES (%s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query3, (int(line[10]), line[1], line[4], line[9]))

In [225]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query3 = "select * from listen_history where song='All Hands Against His Own'"

try:
    rows = session.execute(query3)
except Exception as e:
    print(e)

for row in rows:
    print("\033[1m"+"User ID:"+"\033[0m",row.userid,
          "\033[1m"+"First Name:"+"\033[0m",row.firstname,
          "\033[1m"+"Last Name:"+"\033[0m",row.lastname,
          "\033[1m"+"Song:"+"\033[0m",row.song)

User ID: 29 First Name: Jacqueline Last Name: Lynch Song: All Hands Against His Own
User ID: 80 First Name: Tegan Last Name: Levine Song: All Hands Against His Own
User ID: 95 First Name: Sara Last Name: Johnson Song: All Hands Against His Own


### VERIFYING QUERY 3

In [226]:
df = read_row_data()
df = df[(df.song=='All Hands Against His Own')]
df = df[['userId', 'firstName', 'lastName', 'song']]
df

,userId,firstName,lastName,song
2792,80,Tegan,Levine,All Hands Against His Own
5135,95,Sara,Johnson,All Hands Against His Own
6298,29,Jacqueline,Lynch,All Hands Against His Own


### Drop the tables before closing out the sessions

In [227]:
## TO-DO: Drop the table before closing out the sessions

In [228]:
try:
    query1 = "drop table if exists session_history"
    rows = session.execute(query1)
except Exception as e:
    print(e)

In [229]:
try:
    query2 = "drop table if exists user_history"
    rows = session.execute(query2)
except Exception as e:
    print(e)

In [230]:
try:
    query3 = "drop table if exists listen_history"
    rows = session.execute(query3)
except Exception as e:
    print(e)

### Close the session and cluster connection

In [231]:
session.shutdown()
cluster.shutdown()